In [1]:
from typing import TypedDict

class Person(TypedDict):
    name: str
    age: int
    email: str

person: Person = {
    "name": "Alice",
    "age": 30,
    "email": "alice@example.com"
}

print(person)

{'name': 'Alice', 'age': 30, 'email': 'alice@example.com'}


### GPT Sturcture_Output. 

In [3]:
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
from typing import TypedDict, Optional, Literal, Annotated

load_dotenv()

model = ChatOpenAI()

review = """I recently upgraded to the Samsung Galaxy S24 Ultra, and I must say, it’s an absolute powerhouse! The Snapdragon 8 Gen 3 processor makes everything lightning fast—whether I’m gaming, multitasking, or editing photos. The 5000mAh battery easily lasts a full day even with heavy use, and the 45W fast charging is a lifesaver. The S-Pen integration is a great touch for note-taking and quick sketches, though I don't use it often. What really blew me away is the 200MP camera—the night mode is stunning, capturing crisp, vibrant images even in low light. Zooming up to 100x actually works well for distant objects, but anything beyond 30x loses quality. However, the weight and size make it a bit uncomfortable for one-handed use. Also, Samsung’s One UI still comes with bloatware—why do I need five different Samsung apps for things Google already provides? The $1,300 price tag is also a hard pill to swallow.

Pros:
Insanely powerful processor (great for gaming and productivity)
Stunning 200MP camera with incredible zoom capabilities
Long battery life with fast charging
S-Pen support is unique and useful. """

class format_review(TypedDict):

    summary: Annotated[str, "Write a brief summary of the review."]
    sentiments: Annotated[Literal["pos", "cons"], "Select one of them positive or negative."]
    pros: Annotated[Optional[list[str]], "Write the pros of the review."]
    cons: Annotated[Optional[list[str]], "Write the cons of the review."]
    name: Annotated[Optional[str], "Write the name of the reviewr."]

structure_model = model.with_structured_output(format_review)

result = structure_model.invoke(review)

print(result)

{'summary': 'Highly powerful and feature-packed flagship smartphone with a few drawbacks', 'sentiments': 'pos', 'pros': ['Insanely powerful Snapdragon 8 Gen 3 processor for lightning-fast performance', 'Stunning 200MP camera with excellent zoom capabilities', 'Long-lasting 5000mAh battery with 45W fast charging support', 'Unique and useful S-Pen integration for note-taking and sketches'], 'cons': ['Heavy weight and large size make one-handed use uncomfortable', 'Bloatware in One UI with redundant Samsung apps', 'High price of $1,300', 'Loss of quality in zoom beyond 30x'], 'name': 'Samsung Galaxy S24 Ultra Review'}


### For datavalidation --> Pydantic

* Forcing user to stick with the programmer datatype. 

In [12]:
from pydantic import BaseModel, Field, EmailStr
from typing import Optional

class Student(BaseModel):

    name: str = "Arjun"
    age: Optional[int] = None
    #email: Optional[EmailStr] = None
    cgpa: float = Field(gt = 0, lt = 10, default=5, description="A decimal value shows performance of the student in the class.")

new_student = {'name': 'Jass', 'cgpa': 8}

student = Student(**new_student)

print(student)

student_dict = dict(student)

print(student_dict['cgpa'])

name='Jass' age=None cgpa=8.0
8.0


In [15]:
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
from typing import TypedDict, Optional, Literal, Annotated
from pydantic import BaseModel, Field

load_dotenv()

model = ChatOpenAI()

review = """I recently upgraded to the Samsung Galaxy S24 Ultra, and I must say, it’s an absolute powerhouse! The Snapdragon 8 Gen 3 processor makes everything lightning fast—whether I’m gaming, multitasking, or editing photos. The 5000mAh battery easily lasts a full day even with heavy use, and the 45W fast charging is a lifesaver. The S-Pen integration is a great touch for note-taking and quick sketches, though I don't use it often. What really blew me away is the 200MP camera—the night mode is stunning, capturing crisp, vibrant images even in low light. Zooming up to 100x actually works well for distant objects, but anything beyond 30x loses quality. However, the weight and size make it a bit uncomfortable for one-handed use. Also, Samsung’s One UI still comes with bloatware—why do I need five different Samsung apps for things Google already provides? The $1,300 price tag is also a hard pill to swallow.

Pros:
Insanely powerful processor (great for gaming and productivity)
Stunning 200MP camera with incredible zoom capabilities
Long battery life with fast charging
S-Pen support is unique and useful. """

"""
class format_review(TypedDict):

    summary: Annotated[str, "Write a brief summary of the review."]
    sentiments: Annotated[Literal["pos", "cons"], "Select one of them positive or negative."]
    pros: Annotated[Optional[list[str]], "Write the pros of the review."]
    cons: Annotated[Optional[list[str]], "Write the cons of the review."]
    name: Annotated[Optional[str], "Write the name of the reviewr."]
"""

class Format_review(BaseModel):

    summary: str = Field(description="Write a brief summary of the review.")    
    sentiments: Literal["pos", "neg"] = Field(description="Select one of them positive or negative.")
    pros: list[str] = Field(description="Write the pros of the review.")
    cons: list[str] = Field(description="Write the cons of the review.")
    name: Optional[str] = Field(default="None", description="Write the name of the reviewr.")

structure_model = model.with_structured_output(Format_review)

result = structure_model.invoke(review)

print(result)
print(dict(result))

/Users/arjun/miniforge3/envs/ai/lib/python3.11/site-packages/langchain_openai/chat_models/base.py:2073: UserWarning: Cannot use method='json_schema' with model gpt-3.5-turbo since it doesn't support OpenAI's Structured Output API. You can see supported models here: https://platform.openai.com/docs/guides/structured-outputs#supported-models. To fix this warning, set `method='function_calling'. Overriding to method='function_calling'.
  warnings.warn(


summary='The Samsung Galaxy S24 Ultra is an absolute powerhouse with impressive features but comes with a high price tag and some drawbacks.' sentiments='pos' pros=['Insanely powerful processor (great for gaming and productivity)', 'Stunning 200MP camera with incredible zoom capabilities', 'Long battery life with fast charging', 'S-Pen support is unique and useful'] cons=['Weight and size make it uncomfortable for one-handed use', "Bloatware in Samsung's One UI", 'High price tag of $1,300'] name=None
{'summary': 'The Samsung Galaxy S24 Ultra is an absolute powerhouse with impressive features but comes with a high price tag and some drawbacks.', 'sentiments': 'pos', 'pros': ['Insanely powerful processor (great for gaming and productivity)', 'Stunning 200MP camera with incredible zoom capabilities', 'Long battery life with fast charging', 'S-Pen support is unique and useful'], 'cons': ['Weight and size make it uncomfortable for one-handed use', "Bloatware in Samsung's One UI", 'High pric

### There is another schema called Json schema. You use it when your application have used different languages like for backend python and for frontend Java. 

In [ ]:
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
from typing import TypedDict, Annotated, Optional, Literal
from pydantic import BaseModel, Field

load_dotenv()

model = ChatOpenAI()

# schema
json_schema = {
  "title": "Review",
  "type": "object",
  "properties": {
    "key_themes": {
      "type": "array",
      "items": {
        "type": "string"
      },
      "description": "Write down all the key themes discussed in the review in a list"
    },
    "summary": {
      "type": "string",
      "description": "A brief summary of the review"
    },
    "sentiment": {
      "type": "string",
      "enum": ["pos", "neg"],
      "description": "Return sentiment of the review either negative, positive or neutral"
    },
    "pros": {
      "type": ["array", "null"],
      "items": {
        "type": "string"
      },
      "description": "Write down all the pros inside a list"
    },
    "cons": {
      "type": ["array", "null"],
      "items": {
        "type": "string"
      },
      "description": "Write down all the cons inside a list"
    },
    "name": {
      "type": ["string", "null"],
      "description": "Write the name of the reviewer"
    }
  },
  "required": ["key_themes", "summary", "sentiment"]
}


structured_model = model.with_structured_output(json_schema)

result = structured_model.invoke("""I recently upgraded to the Samsung Galaxy S24 Ultra, and I must say, it’s an absolute powerhouse! The Snapdragon 8 Gen 3 processor makes everything lightning fast—whether I’m gaming, multitasking, or editing photos. The 5000mAh battery easily lasts a full day even with heavy use, and the 45W fast charging is a lifesaver.

The S-Pen integration is a great touch for note-taking and quick sketches, though I don't use it often. What really blew me away is the 200MP camera—the night mode is stunning, capturing crisp, vibrant images even in low light. Zooming up to 100x actually works well for distant objects, but anything beyond 30x loses quality.

However, the weight and size make it a bit uncomfortable for one-handed use. Also, Samsung’s One UI still comes with bloatware—why do I need five different Samsung apps for things Google already provides? The $1,300 price tag is also a hard pill to swallow.

Pros:
Insanely powerful processor (great for gaming and productivity)
Stunning 200MP camera with incredible zoom capabilities
Long battery life with fast charging
S-Pen support is unique and useful

""")

print(result)

### Some models do not support structured output schema. So, there you will have to use Output Parsers. 